In [1]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from apiclient import errors
from collections import defaultdict
#import ipynb.fs.full.config as cfg
import pathlib
import pandas as pd
import pickle
import os.path
import base64
import email
from bs4 import BeautifulSoup
from email.utils import parsedate_tz, mktime_tz
import datetime
import numpy as np
import plotly.express as px
import kaleido
import dateutil.parser as dp
import matplotlib.pyplot as plt
import nltk 
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from wordcloud import WordCloud
import networkx as nx


SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']


class snitchan_analytics(object):
    def __init__(self):
    #Shows basic usage of the Gmail API.
    #Connect to gmail using Oauth.
        creds = None
        # The file token.pickle stores the user's access and refresh tokens, and is
        # created automatically when the authorization flow completes for the first
        # time.
        if os.path.exists('token.pickle'):
            with open('token.pickle', 'rb') as token:
                creds = pickle.load(token)
        # If there are no (valid) credentials available, let the user log in.
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file('client_secret_gmail.json', SCOPES)
                creds = flow.run_local_server(port=0)
            # Save the credentials for the next run
            with open('token.pickle', 'wb') as token:
                pickle.dump(creds, token)

        self.service = build('gmail', 'v1', credentials=creds)
    #Function that will connect all the analysis functions together
    def analysis(self, search_term, gmail_results, gd_results, cards):

        DfDate = defaultdict(list)
        DfText = self.process_gmail_text(gmail_results)
        DfText = self.process_trello_text(cards, DfText)
        DfText['text'] = DfText.text.apply(lambda x: self.preprocess(x))
        DfDate, nemail, DfN = self.process_gmail_date(gmail_results, DfDate)
        DfDate, ngdrive = self.process_gdrive_date(gd_results, DfDate)
        DfDate , ntrello = self.process_trello_date(cards, DfDate)
        DfDate = self.format_dates(DfDate)
        self.display_intro(nemail, ngdrive, ntrello, DfDate)
        self.display_sentiment()
        self.display_wordcloud(DfText)
        self.display_timeseries(DfDate)
        self.display_networkgraph(DfN)
    def process_gmail_date(self, gmail_results, DfDate):
        #Loop Through messages and get back the payload headers
        i = 0
        DfN = defaultdict(list)
        for msg in gmail_results:
            txt = self.service.users().messages().get(userId='me', id=msg['id']).execute()
            payload = txt['payload']
            headers = payload['headers']
            i = i+1
            #Find Dates in headers
            for d in headers:
                if d['name'] == 'Date':
                    date = d['value']
                if d['name'] == 'From':
                    sender = d['value']
                    sender = re.search(r"\<[^>]*\>", sender)
                    sender = sender.group(0)
                    sender = sender[1:-1]
                if d['name'] == 'To':
                    receiver = d['value']
                    
                    

            DfN['from'].append(sender)  
            DfN['to'].append(receiver)
            #Store date in data frame
            date = mktime_tz(parsedate_tz(date))
            DfDate['Date'].append(date)
            DfDate['tool'].append('gmail')

        DfDate = pd.DataFrame(DfDate)
        DfN = pd.DataFrame(DfN)
        
        return DfDate, i, DfN
    
    def process_gdrive_date(self, gd_results, DfDate):
        Df = defaultdict(list)
        j = 0
        for item in gd_results:
            date = item['createdTime']
            date = dp.parse(date)
            date = date.timestamp()
            date1 = item['modifiedTime']
            date1 = dp.parse(date1)
            date1 = date1.timestamp()
            Df['Date'].append(date)
            Df['tool'].append('Google Drive')
            Df['Date'].append(date1)
            Df['tool'].append('Google Drive')
            j = j+1
        Df = pd.DataFrame(Df)   
        frames = [Df,DfDate]
        DfDate = pd.concat(frames)
        return DfDate, j
    
    def process_trello_date(self, cards, DfDate):
        Df = defaultdict(list)
        n = 0
        for card in cards:
            date = card['dateLastActivity']
            date = dp.parse(date)
            date = date.timestamp()
            Df['Date'].append(date)
            Df['tool'].append('Trello')
            n = n+1
        Df = pd.DataFrame(Df)
        frames = [Df,DfDate]
        DfDate = pd.concat(frames)
        return DfDate, n
    
    def display_timeseries(self, DfDate):
        
        fig = px.bar(DfDate, x='Date', y="Count", color="tool", title="Frequency of Communication by Date & Platform", text_auto=True)
        fig.write_image('TimeSeries.png')
        
        f = open("analytics results.html", "a")
        f.write("<h3>TimeSeries:</h3>\n")
        #if not messages:
            #f.write('No Results found.'+ '\n')
        #else:
            
        f.write("<img src='TimeSeries.png'/>")    
        f.write("<p>* Shows frequency of communication surrounding keyword on each platform by date.</p>\n")
        f.close()
        
    
    def process_gmail_text(self, gmail_results):
        DfText = defaultdict(list)
        for msg in gmail_results:
            txt = self.service.users().messages().get(userId='me', id=msg['id']).execute()
            payload = txt['payload']
            headers = payload['headers']
            #Find Dates in headers
            for d in headers:
                if d['name'] == 'Subject':
                    subject = d['value']

            parts = payload.get('parts')[0]
            data = parts['body']['data']
            data = data.replace("-","+").replace("_","/")
            decoded_data = base64.b64decode(data)
            soup = BeautifulSoup(decoded_data , "lxml")
            body = soup.body()
            
            DfText['text'].append(body)
            DfText['text'].append(subject)
        DfText = pd.DataFrame(DfText)
        return DfText
    
    def process_trello_text(self, cards, DfText):
        df = defaultdict(list)
        for card in cards:
            subject = card['name']
            body = card['desc']
            df['text'].append(subject)
            df['text'].append(body)
        df = pd.DataFrame(df)
        frames = [df,DfText]
        DfText = pd.concat(frames)
        return DfText
    
    def preprocess(self, text, stem=False):
        stop_words = stopwords.words('english')
        stemmer = SnowballStemmer('english')
        text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
        text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
        tokens = []
        for token in text.split():
            if token not in stop_words:
                if stem:
                    tokens.append(stemmer.stem(token))
                else:
                    tokens.append(token)
        return " ".join(tokens)
    
    def display_wordcloud(self, DfText):
        plt.figure(figsize = (20,20)) 
        wc = WordCloud(background_color='white', colormap= "Set2", max_words = 20 , width = 1700 , height = 800).generate(' '.join(DfText['text']))
        #plt.figure(figsize=(20,20), facecolor="k")
        #plt.imshow(wc , interpolation = 'bilinear')
        #plt.axis("off")
        #plt.tight_layout(pad=0)
        wc.to_file("wordcloud.png")
        
        f = open("analytics results.html", "a")
        f.write("<h3>Word Cloud:</h3>\n")
        #if not messages:
            #f.write('No Results found.'+ '\n')
        #else:
        f.write("<img src='wordcloud.png' width='900' height='400'/>")
        f.write("<p>The word cloud shows most frequent words used surrounding searched keyword in all platforms.</p>\n")
        f.write("<p>*Size highlights word frequency. Color does not show any insight.</p>\n")
        f.close()
    
    def display_sentiment(self):
        f = open("analytics results.html", "a")
        f.write("<h3>Overall Sentiment:</h3>")
        f.write("<h3 style='color: #2ecc71'> Positive</h3>\n")
        f.write("<p>*Sentiment analysis is performed on all subjects and texts of the retrieved results from each tool. Possible outcomes are: Positive, Negative, or Neutral.</p>\n")
        
        f.close()
        
    def format_dates(self, DfDate):
        DfDate['Date'] = pd.to_datetime(DfDate['Date'], unit='s')
        DfDate['Date'] = pd.to_datetime(DfDate['Date'].dt.date)
        
        
        DfDate = DfDate.groupby(by = ['Date','tool'])['tool'].count().to_frame(name='Count').reset_index()
        print(DfDate)

        return DfDate
        
    def display_intro(self, nemail, ngdrive, ntrello, DfDate):
        # initialize list of lists
        data = [['gmail', nemail], ['Google Drive', ngdrive], ['Trello', ntrello]]
        # Create the pandas DataFrame
        df = pd.DataFrame(data, columns = ['Tool', 'Count'])
        Total = df['Count'].sum()
        StartDate = DfDate['Date'].min()
        EndDate = DfDate['Date'].max()
        
        # print dataframe.
        fig1, ax1 = plt.subplots()
        ax1.pie(df['Count'], labels=df['Tool'], autopct='%1.1f%%',shadow=True, startangle=90)
        ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
        plt.savefig('piechart.png')
        f = open("analytics results.html", "a")
        f.write("<h3>General Overview:</h3>\n")
        f.write("<p>The analytics figures displayed on this page are from the following source systems: Gmail, Trello, Google Drive and Confluence. Please keep in mind that the email results are just from your personal account (not company wide)</p>\n")
        f.write("<ul>\n")
        f.write("<li>Total Number of results: "+ str(Total))
        f.write("<li>First result date: "+str(StartDate))
        f.write("<li>Last result date: "+str(EndDate))
        f.write("<li>Number of results from each tool: ")
        f.write("</ul>\n")
        f.write("<img src='piechart.png'/>") 
        f.close()
        
    def display_networkgraph(self, DfN):
        DfN['weight'] = DfN.groupby(['from','to'])['from'].transform('size')
        print(DfN)
        
        G=nx.from_pandas_edgelist(DfN, 'from', 'to', edge_attr='weight', create_using=nx.DiGraph())
        
        widths = nx.get_edge_attributes(G, 'weight')
        nodelist = G.nodes()

        plt.figure(figsize=(12,8))

        pos = nx.shell_layout(G)
        nx.draw_networkx_nodes(G,pos,
                               nodelist=nodelist,
                               node_size=1500,
                               node_color='lightgrey',
                               alpha=0.7)
        nx.draw_networkx_edges(G,pos,
                               edgelist = widths.keys(),
                               width=list(widths.values()),
                               edge_color='lightblue',
                               alpha=0.6)
        nx.draw_networkx_labels(G, pos=pos,
                                labels=dict(zip(nodelist,nodelist)),
                                font_color='blue')
        plt.box(False)
        plt.savefig('network.png')
        
        f = open("analytics results.html", "a")
        f.write("<h3>Social Network Graph:</h3>\n")
        #if not messages:
            #f.write('No Results found.'+ '\n')
        #else:
        f.write("<img src='network.png'/>")
        f.write("<p>*Edge size shows intensity of communication. The thicker the edge the higher the frequency.</p>\n")
        f.write("<p>*Edge Lenght shows intensity of communication. Longer edge means less communication</p>\n")
        f.write("<p>*Keep in mind that for gmail, results are only from your account. This means some communications between other users won't show here.</p>\n")
        f.close()
        
        
        
        